In [23]:
%matplotlib inline
import numpy as np
import pandas as pd

# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec, Word2Vec

# Setup nlkt
import nltk.data
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

import string
import os
import sys
import re

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [24]:
def encode_reviews(text):
    encode = [w for w in word_tokenize(text) if not w.lower() in stopset]
    encode = " ".join(WordNetLemmatizer().lemmatize(lem_item, pos='n') for lem_item in encode)
    # delete special characters except for apostraphes
    encode = str(encode).translate(string.maketrans("",""), string.punctuation.replace("'",""))
    encode = encode.replace("''", "")
    encode = encode.replace(" ' ", "")
    encode = encode.replace("' ", "")
    # remove multipe spaces
    encode = re.sub( '\s+', ' ', encode).strip()
    encode = encode.lower()
    return encode

In [25]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
stopset = set(stopwords.words('english'))

reviews = pd.DataFrame(columns=["reviews", "sentiment"])

sentiment_vec = ["neg", "pos"]

base_directory = r"C:\Github\springboard_cap1\bayes\data"

for item in sentiment_vec:
    directory = base_directory + "\\" + item
    num_files = len(os.walk(directory).next()[2])
    for count, filename in enumerate(os.listdir(directory)):
        print 'Reading {0} review number {1} of {2}\r'.format(item, count, num_files),
        if filename.endswith('.txt'):
            filename_loc = directory + "\\" + filename
            with open(filename_loc) as f:
                paragraph = f.read().strip().decode('utf8')
                # remove HTML tags
                paragraph = re.sub('<[^<]+?>', ' ', paragraph)
                # remove numbers
                paragraph = re.sub(r'\d+', '', paragraph)
                #sentence_split = paragraph sent_detector.tokenize(paragraph)
                #for sentence_count, sentence_item in enumerate(sentence_split):
                #    reviews.loc[filename + " " + str(sentence_count), "reviews"] = encode_reviews(sentence_item)
                #    reviews.loc[filename + " " + str(sentence_count), "sentiment"] = item
                reviews.loc[filename, "reviews"] = encode_reviews(paragraph)
                reviews.loc[filename, "sentiment"] = item
            continue
        else:
            continue

In [26]:
critics = pd.read_csv('./critics.csv')
#let's drop rows with missing quotes
critics = critics[~critics.quote.isnull()]
critics.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
1,Derek Adams,fresh,114709,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559,Toy story
2,Richard Corliss,fresh,114709,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559,Toy story
3,David Ansen,fresh,114709,Newsweek,A winning animated feature that has something ...,2008-08-18,9559,Toy story
4,Leonard Klady,fresh,114709,Variety,The film sports a provocative and appealing st...,2008-06-09,9559,Toy story
5,Jonathan Rosenbaum,fresh,114709,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559,Toy story


In [27]:
new_pd_index = critics.apply(lambda row: str(row["imdb"]) + " "+ str(row["critic"]), axis=1)
rt_set = pd.DataFrame(index=new_pd_index)
# remove tags
rt_set["reviews"] = np.asarray(critics["quote"].str.replace("</", " "))
# remove numbers
rt_set["reviews"] = np.asarray(rt_set.apply(lambda x: re.sub(r'\d+', '', x["reviews"]), axis=1))
# encode
rt_set["reviews"] = np.asarray(rt_set.apply(lambda x: encode_reviews(x["reviews"]), axis=1))
rt_set["sentiment"] = np.asarray(critics["fresh"].replace(["fresh", "rotten"], ["pos", "neg"] ))
combined_set = pd.concat([reviews, rt_set])

In [28]:
reviews.to_csv('imdb_set.csv')
combined_set.to_csv('train_set.csv')
rt_set.to_csv('rt_set.csv')